<a href="https://colab.research.google.com/github/idutsu/kirikuchikun-ai/blob/main/src/search-word-wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 必要なライブラリをインストール
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3 gensim tqdm
!pip install unidic
!python -m unidic download

import pandas as pd
import json
import MeCab
from gensim.models import KeyedVectors

# インデックスCSVファイルのパス
index_csv_path = "/content/drive/MyDrive/kirikuchikun-ai/wikipedia_json_output/index.csv"
# Word2Vecモデルのパス
word2vec_model_path = "/content/drive/MyDrive/kirikuchikun-ai/jawiki.all_vectors.200d.txt"

# Word2Vecモデルをロード
print("Word2Vecモデルをロード中...")
word_vectors = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=False)
print("Word2Vecモデルのロードが完了しました。")

# MeCabの初期化
mecab = MeCab.Tagger()

def extract_words(text):
    """
    テキストを形態素解析して名詞、形容詞、動詞、副詞を抽出する。
    """
    allowed_pos = ["名詞", "形容詞", "動詞", "副詞"]
    parsed = mecab.parse(text).splitlines()
    words = []
    for line in parsed:
        if "\t" in line:
            surface, features = line.split("\t")
            pos = features.split(",")[0]  # 品詞
            if pos in allowed_pos:
                words.append(surface)
    return words

def calculate_similarity(word, target_word, word_vectors):
    """
    Word2Vecで類似度を計算する。
    """
    try:
        return word_vectors.similarity(word, target_word)
    except KeyError:
        return None  # モデルに単語が存在しない場合

# インデックスCSVを読み込む
df = pd.read_csv(index_csv_path)



In [ ]:

# 検索したいタイトルとターゲット単語
search_title = "手錠"
target_word = "手錠"

# タイトルが完全一致する行を検索
matched_rows = df[df['title'] == search_title]

if not matched_rows.empty:
    # 一致するタイトルが見つかった場合
    for _, row in matched_rows.iterrows():
        file_path = row['file_path']
        print(f"タイトル「{search_title}」が見つかりました。ファイルパス: {file_path}")

        # 対応するJSONファイルを読み込む
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    data = json.loads(line)
                    if data.get('title') == search_title:
                        text = data.get('text', "")
                        print("テキスト取得完了。形態素解析中...\n")

                        # 形態素解析で単語を抽出
                        extracted_words = extract_words(text)
                        print(f"抽出された単語数: {len(extracted_words)}")

                        # Word2Vecで類似度を計算
                        print(f"\n「{target_word}」との類似度が高い単語:")
                        for word in extracted_words:
                            similarity = calculate_similarity(word, target_word, word_vectors)
                            if similarity is not None and similarity > 0.3:  # しきい値0.3
                                print(f"{word}: {similarity:.4f}")
                        break  # 完全一致したタイトルの処理が完了したら終了
        except Exception as e:
            print(f"ファイルの読み込み中にエラーが発生しました: {e}")
else:
    print(f"タイトル「{search_title}」はインデックスに存在しません。")